In [ ]:
import pandas as pd
import math
from openpyxl.styles import Alignment, Font, PatternFill, Border, Side

In [ ]:
path='/content/Datawifi_Trafico_Hora_1749457620336.csv'
datos=pd.read_csv(path)
datos.shape

(563903, 13)

In [ ]:
path2='/content/GEN_Sitios_Activos_SI-2025-06-05.xlsx'
datos2=pd.read_excel(path2)
datos2.shape

(6415, 46)

In [ ]:

datos=datos[['FECHA','HORA','INPUT','OUTPUT','NOMBRE_ZONA']]


In [ ]:
#Dividir los datos de una conlumna y guardarlos en columnas diferentes
datos[['ID BENEFICIARIO','NOMBRE_ZONA']]=datos['NOMBRE_ZONA'].str.split('_',expand=True)

In [ ]:
#pasar un dato de tipo objeto a tipo entero
datos['ID BENEFICIARIO']=datos['ID BENEFICIARIO'].astype(int)

In [ ]:
datos=datos.merge(datos2 [['Id_Beneficiario','Nombre_Sede','Centro_Poblado','Meta','Departamento','Municipio','Estado']],left_on='ID BENEFICIARIO',right_on='Id_Beneficiario',how='inner')

In [ ]:
#renombrar columnas
datos=datos.rename(columns={'INPUT':'UPLOAD MB', 'OUTPUT':'DOWNLOAD MB', 'Meta':'GRUPO', 'Centro_Poblado':'CENTRO POBLADO', 'Nombre_Sede':'NOMBRE SEDE', 'Departamento':'DEPARTAMENTO', 'Municipio':'MUNICIPIO'})

In [ ]:
#Agregar columna y llenarla con datos
datos['CONSOLIDADO MB']=datos['UPLOAD MB']+datos['DOWNLOAD MB']

In [ ]:
#Organizar los datos
datos=datos.sort_values(by=['ID BENEFICIARIO','FECHA','HORA'])

In [ ]:
#Reemplazar datos
datos['NOMBRE_ZONA']=datos['NOMBRE_ZONA'].replace({None:'Indoor','outdoor':'Outdoor'})

In [ ]:
datos=datos[['ID BENEFICIARIO','GRUPO','DEPARTAMENTO','MUNICIPIO','CENTRO POBLADO','NOMBRE SEDE','FECHA','HORA','UPLOAD MB','DOWNLOAD MB','CONSOLIDADO MB','NOMBRE_ZONA','Estado']]

In [ ]:
datos_tabla=datos.copy()

In [ ]:
#Organizar columnas
datos=datos[['ID BENEFICIARIO','GRUPO','DEPARTAMENTO','MUNICIPIO','CENTRO POBLADO','NOMBRE SEDE','FECHA','HORA','UPLOAD MB','DOWNLOAD MB','CONSOLIDADO MB','NOMBRE_ZONA']]

In [ ]:
#Crear un dataframe sin los datos del grupo 3a (~)se utiliza para negar
datos_tabla=datos_tabla[datos_tabla['Estado'].isin(['EN OPERACIÓN'])]
datos_tabla.shape

(557148, 13)

In [ ]:
tabla1=datos_tabla.groupby('DEPARTAMENTO').agg(
    DOWNLOAD_MB=('DOWNLOAD MB','sum'),
    UPLOAD_MB=('UPLOAD MB','sum'),
    CONSOLIDADO_MB=('CONSOLIDADO MB','sum')
).reset_index()

In [ ]:
#Sumar las columnas para total
totales=tabla1[['DOWNLOAD_MB','UPLOAD_MB','CONSOLIDADO_MB']].sum()
#crear una fila en la tabla
df_totales=pd.DataFrame([['Total General',totales['DOWNLOAD_MB'],totales['UPLOAD_MB'],totales['CONSOLIDADO_MB']]],columns=tabla1.columns)
tabla1_final=pd.concat([tabla1,df_totales],ignore_index=True)

columnas_a_formatear=['DOWNLOAD_MB','UPLOAD_MB','CONSOLIDADO_MB']
for col in columnas_a_formatear:
    tabla1_final[col]=tabla1_final[col].apply(lambda x: f'{x:,.0f}'.replace(',','.'))

In [ ]:
datos_indoor=datos_tabla[(datos_tabla['NOMBRE_ZONA']=='Indoor')]

dinamica_indoor=datos_indoor.groupby('DEPARTAMENTO').agg(INDOOR_MB=('CONSOLIDADO MB','sum')).reset_index()

In [ ]:
datos_outdoor=datos_tabla[(datos_tabla['NOMBRE_ZONA']=='Outdoor')]

dinamica_outdoor=datos_outdoor.groupby('DEPARTAMENTO').agg(OUTDOOR_MB=('CONSOLIDADO MB','sum')).reset_index()

In [ ]:
tabla2=dinamica_indoor.merge(dinamica_outdoor,on='DEPARTAMENTO',how='left')
tabla2['CONSOLIDADO (MB)']=tabla2['INDOOR_MB']+tabla2['OUTDOOR_MB']
tabla2=tabla2.rename(columns={'INDOOR_MB':'INDOOR (MB)','OUTDOOR_MB':'OUTDOOR (MB)'})

In [ ]:
#Sumar las columnas para total
totales=tabla2[['INDOOR (MB)','OUTDOOR (MB)','CONSOLIDADO (MB)']].sum()
#crear una fila en la tabla
df_totales=pd.DataFrame([['Total General',totales['INDOOR (MB)'],totales['OUTDOOR (MB)'],totales['CONSOLIDADO (MB)']]],columns=tabla2.columns)
tabla2_final=pd.concat([tabla2,df_totales],ignore_index=True)

columnas_a_formatear=['INDOOR (MB)','OUTDOOR (MB)','CONSOLIDADO (MB)']
for col in columnas_a_formatear:
    tabla2_final[col]=tabla2_final[col].apply(lambda x: f'{x:,.0f}'.replace(',','.'))

In [ ]:
tabla3=datos_tabla.groupby('HORA').agg(
    DOWNLOAD_MB=('DOWNLOAD MB','sum'),
    UPLOAD_MB=('UPLOAD MB','sum'),
    CONSOLIDADO_MB=('CONSOLIDADO MB','sum')
).reset_index()

In [ ]:
#Sumar las columnas para total
totales=tabla3[['DOWNLOAD_MB','UPLOAD_MB','CONSOLIDADO_MB']].sum()
#crear una fila en la tabla
df_totales=pd.DataFrame([['Total General',totales['DOWNLOAD_MB'],totales['UPLOAD_MB'],totales['CONSOLIDADO_MB']]],columns=tabla3.columns)
tabla3_final=pd.concat([tabla3,df_totales],ignore_index=True)

columnas_a_formatear=['DOWNLOAD_MB','UPLOAD_MB','CONSOLIDADO_MB']
for col in columnas_a_formatear:
    tabla3_final[col]=tabla3_final[col].apply(lambda x: f'{x:,.0f}'.replace(',','.'))

In [ ]:
datos_tabla['FECHA']=pd.to_datetime(datos_tabla['FECHA'],format='%d/%m/%Y')
#fecha_formateada=fecha_creacion.strftime('%Y%m%d')

fecha_creacion=datos_tabla['FECHA'].iloc[0]

meses={
    1: 'ENE',2: 'FEB',3: 'MAR',4: 'ABR',
    5: 'MAY',6: 'JUN',7: 'JUL',8: 'AGO',
    9: 'SEP',10: 'OCT',11: 'NOV',12: 'DIC'
}

nombre_mes=meses[fecha_creacion.month]
anio=fecha_creacion.strftime('%y')
anio_mes=f'{nombre_mes}-{anio}'
print(anio_mes)

MAY-25


In [ ]:
#Número de registros por hoja
registros_por_hoja=1000000

#Calcular el número de hojas que se necesitan segun el valor
num_hojas=math.ceil(len(datos)/registros_por_hoja)

#Crear el archivo de excel y agregar la hoja resumen
with pd.ExcelWriter(f'{anio_mes} Trafico_Detallado.xlsx', engine='openpyxl') as writer:
    #Crear la hoja de resumen
    wb=writer.book
    ws_resumen=wb.create_sheet(title='RESUMEN')

    #Establecer color de fondo
    fill_white=PatternFill(start_color='FFFFFF',end_color='FFFFFF',fill_type='solid')
    #Establecer color de titulos
    fill_blue=PatternFill(start_color='ADD8E6',end_color='ADD8E6',fill_type='solid')

    #Crear fuente en negrita
    bold_font=Font(bold=True)

    #Crear bordes
    thin_border=Border(
        left=Side(border_style='thin', color='D3D3D3'),
        right=Side(border_style='thin', color='D3D3D3'),
        top=Side(border_style='thin', color='D3D3D3'),
        bottom=Side(border_style='thin', color='D3D3D3')
    )

    #Funcion para aplicar fondo azul claro a las celdas B3 a la D3
    def apply_blue_background_to_b3(ws):
        for row in [3,25,47]:
            for col in ['B','C','D','E']:
                ws[f'{col}{row}'].fill=fill_blue

    def apply_white_background(ws):
        max_row=200
        max_col=100
        for row in ws.iter_rows(min_row=1,max_row=max_row, min_col=1,max_col=max_col):
            for cell in row:
                cell.fill=fill_white
                cell.alignment=Alignment(horizontal='center',vertical='center')
                cell.border=None


    def adjust_columns_b_to_e(ws):
        adjust_columns=['B','C','D','E']
        for column in adjust_columns:
            max_length=0
            columns_cells=ws[column]
            for cell in columns_cells:
                try:
                    if len(str(cell.value))>max_length:
                        max_length=len(cell.value)
                except:
                    pass

            adjusted_width=(max_length+6)
            ws.column_dimensions[column].width=adjusted_width

    #adjust_columns_b_to_e(ws_resumen)

    def apply_bold_to_specific_cells(ws):
        cells_to_bold=['B20','B42','B72']
        for cell in cells_to_bold:
            ws[cell].font=bold_font

    def apply_gray_borders(ws):
        for row in ws.iter_rows(min_row=3,max_row=20,min_col=2,max_col=5):
            for cell in row:
                cell.border=thin_border
        for row in ws.iter_rows(min_row=25,max_row=42,min_col=2,max_col=5):
            for cell in row:
                cell.border=thin_border
        for row in ws.iter_rows(min_row=47,max_row=72,min_col=2,max_col=5):
            for cell in row:
                cell.border=thin_border


    tabla1_final.to_excel(writer,sheet_name='RESUMEN',index=False,startrow=2,startcol=1)
    tabla2_final.to_excel(writer,sheet_name='RESUMEN',index=False,startrow=24,startcol=1)
    tabla3_final.to_excel(writer,sheet_name='RESUMEN',index=False,startrow=46,startcol=1)

    #llamado de funciones
    apply_white_background(ws_resumen)
    apply_blue_background_to_b3(ws_resumen)
    adjust_columns_b_to_e(ws_resumen)
    apply_bold_to_specific_cells(ws_resumen)
    apply_gray_borders(ws_resumen)


    for i in range(num_hojas):
        start_idx=i*registros_por_hoja
        end_idx=(i+1)*registros_por_hoja
        fragmento=datos.iloc[start_idx:end_idx]

        hoja_nombre=f'{anio_mes} Tráfico Detallado{i+1}'

        fragmento.to_excel(writer,sheet_name=hoja_nombre,index=False)

print('archivo exportado correctamente')

archivo exportado correctamente


In [ ]:
# Descargar el archivo a tu máquina local
from google.colab import files
files.download(f'{anio_mes} Trafico_Detallado.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>